In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import sklearn
import sqlite3 as sql 
from sklearn.model_selection import train_test_split
import pickle
import joblib

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.datasets import make_regression
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Ridge
from sklearn.svm import SVR
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import LinearRegression
from sklearn import tree
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.neighbors import NearestNeighbors
from sklearn.ensemble import GradientBoostingRegressor

In [2]:
#create connection to database
database = 'diamonds_train.db'
connection = sql.connect(database)

In [3]:
print(pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", connection))

                     name
0     diamonds_dimensions
1  diamonds_transactional
2     diamonds_properties
3            diamonds_cut
4          diamonds_color
5        diamonds_clarity
6           diamonds_city


In [4]:
cur = connection.cursor()
cur.description
cur

In [5]:
df = pd.DataFrame(cur.fetchall())
df

""


In [6]:
cur.description

In [7]:
query = '''SELECT * FROM diamonds_dimensions'''
query2 = '''SELECT * FROM diamonds_transactional'''
query3 = '''SELECT * FROM diamonds_properties'''
query4 = '''SELECT * FROM diamonds_cut'''
query5 = '''SELECT * FROM diamonds_color'''
query6 = '''SELECT * FROM diamonds_clarity'''
query7 = '''SELECT * FROM diamonds_city'''

In [8]:
df = pd.read_sql_query(query, connection)
df1 = pd.read_sql_query(query2, connection)
df2 = pd.read_sql_query(query3, connection)
df3 = pd.read_sql_query(query4, connection)
df4 = pd.read_sql_query(query5, connection)
df5 = pd.read_sql_query(query6, connection)
df6 = pd.read_sql_query(query7, connection)

In [9]:
df_unido1 = pd.merge(df, df1, on = 'index_id')
df_unido2 = pd.merge(df_unido1,df2, on = 'index_id')
df_unido3 = pd.merge(df_unido2,df3, on = 'cut_id')
df_unido4 = pd.merge(df_unido3,df4, on = 'color_id')
df_unido5 = pd.merge(df_unido4,df5, on = 'clarity_id')
df_unido6 = pd.merge(df_unido5,df6, on = 'city_id')

In [10]:
df_train = df_unido6.loc[:, ['depth', 'table', 'x', 'y', 'z', 'price', 'carat', 'cut', 'color', 'clarity', 'city']]
df_train

,depth,table,x,y,z,price,carat,cut,color,clarity,city
0,62.4,58.0,6.83,6.79,4.25,4268,1.21,Premium,J,VS2,Dubai
1,61.6,58.0,6.40,6.35,3.93,3513,1.02,Premium,J,VS2,Dubai
2,62.3,58.0,5.86,5.80,3.63,1792,0.77,Premium,J,VS2,Dubai
3,59.6,60.0,7.58,7.48,4.49,7553,1.51,Premium,J,VS2,Dubai
4,60.2,62.0,5.40,5.33,3.23,1176,0.57,Premium,J,VS2,Dubai
...,...,...,...,...,...,...,...,...,...,...,...
40450,62.2,54.0,5.24,5.27,3.27,2729,0.54,Ideal,F,IF,Surat
40451,61.9,54.0,5.22,5.25,3.24,2802,0.53,Ideal,F,IF,Surat
40452,62.3,55.0,4.30,4.34,2.69,886,0.30,Ideal,F,IF,Surat
40453,60.9,55.0,4.15,4.23,2.55,768,0.26,Ideal,F,IF,Surat


In [11]:
df_test = pd.read_csv('diamonds_test.csv')
df_test

,id,carat,cut,color,clarity,depth,table,x,y,z,city
0,0,0.79,Very Good,F,SI1,62.7,60.0,5.82,5.89,3.67,Amsterdam
1,1,1.20,Ideal,J,VS1,61.0,57.0,6.81,6.89,4.18,Surat
2,2,1.57,Premium,H,SI1,62.2,61.0,7.38,7.32,4.57,Kimberly
3,3,0.90,Very Good,F,SI1,63.8,54.0,6.09,6.13,3.90,Kimberly
4,4,0.50,Very Good,F,VS1,62.9,58.0,5.05,5.09,3.19,Amsterdam
...,...,...,...,...,...,...,...,...,...,...,...
13480,13480,0.57,Ideal,E,SI1,61.9,56.0,5.35,5.32,3.30,Amsterdam
13481,13481,0.71,Ideal,I,VS2,62.2,55.0,5.71,5.73,3.56,New York City
13482,13482,0.70,Ideal,F,VS1,61.6,55.0,5.75,5.71,3.53,Tel Aviv
13483,13483,0.70,Very Good,F,SI2,58.8,57.0,5.85,5.89,3.45,Surat


In [12]:
cat_features = ['cut', 'color', 'clarity', 'city']
num_features = ['carat', 'table', 'depth', 'x', 'y', 'z']
target = ['price']

In [13]:
df_train[cat_features] = df_train[cat_features].astype('category')
df_test[cat_features] = df_test[cat_features].astype('category')

In [14]:
cat_df = pd.get_dummies(df_train[cat_features])
num_df = df_train
train_df = pd.concat([cat_df, num_df], axis = 1)

cat_df = pd.get_dummies(df_test[cat_features])
num_df = df_test
test_df = pd.concat([cat_df, num_df], axis = 1)

features = list(cat_df.columns) + list(num_df.columns)

In [15]:
scaler = StandardScaler()
X = scaler.fit_transform(train_df.loc[:, features].values.round(3))
y = df_train[target]

In [16]:
model = RandomForestRegressor()
#model_2 = GradientBoostingRegressor(random_state=42)

In [18]:
model.fit(X=X, y=y)
#model_2.fit(X=X, y=y)
#model_3.fit(X=X, y=y)
#model_4.fit(X=X, y=y)
#model_5.fit(X=X, y=y)

/Users/jorgediazserrano/miniconda3/envs/proyectm3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


RandomForestRegressor()

In [19]:
X_test = scaler.transform(test_df.loc[:, features].values)
y_pred = model.predict(X_test).clip(0, 30000)
submission = pd.DataFrame({'id': df_test['id'], 'price': y_pred})
submission.to_csv('new_submission.csv', index = False)

In [20]:
submission

,id,price
0,0,3005.75
1,1,5351.24
2,2,9137.05
3,3,4209.88
4,4,1713.72
...,...,...
13480,13480,1716.51
13481,13481,2413.29
13482,13482,2865.41
13483,13483,2172.86
